**Связь docker контейнера и файловой системы хоста**

# Volume

Это файлы которые лежат на хосте и не зависят от конкретного образа, но зависят от docker в целом. **Основное предназначение** - хранение данных.

### Комманды `volume`

Любая комманда для работы с volume начитается с `docker volume`.

- `ls` - отобразить доступные volumes;
- `rm` - удалить volume;

### Тонкости

Некоторые контейнеры, когда поднимаются создают себе volume автоматически. Потому в один момент можно обнаружить, что у тебя весь диск забит. Например `yandex/clickhouse-server`.

In [ ]:
%%bash
docker run -d --name db_1 yandex/clickhouse-server

# Bind mount

Можно примонтировать к контейнеру любую папку или файл в любом месте на хосте. **Основное предназначение** - входная информация по типу ключей.